In [1]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# Overview

In the following demos we illustrate how to compute the Singular Value Decomposition (SVD) of the gradient of the flow map $ \nabla F_{t_0}^t(\mathbf{x}_0) $ on a general two dimensional unsteady velocity dataset. In our case we chose the ABC which is used throughout the demos on advective barriers in 3D. The notebook is structured as follows:

1. Import data from the file 'ABCunsteady.mat' stored in the folder 'data/ABC'.
2. Define computational parameters and data.
3. Interpolate velocity from (discrete) gridded data.
4. Compute gradient of flow map $ \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0) $.
5. SVD on $ \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0) $

# Import data

In [2]:
# import scipy
import scipy.io as sio

# Import velocity data from file in data-folder
mat_file = sio.loadmat('../../../data/ABC/ABCunsteady.mat')

U = mat_file['u'] # array (NY, NX, NZ)
V = mat_file['v'] # array (NY, NX, NZ)
W = mat_file['w'] # array (NY, NX, NZ)
x = mat_file['x'] # array (1, NX)
y = mat_file['y'] # array (1, NY)
z = mat_file['z'] # array (1, NZ)
time_data = mat_file['t']

# Computational parameters and data

Here we define the computational parameters and the data.

In [3]:
# import numpy
import numpy as np

# Number of cores for parallel computing
Ncores = 7 # int

# Periodic boundary conditions
periodic_x = True # bool
periodic_y = True # bool
periodic_z = True # bool
periodic = [periodic_x, periodic_y, periodic_z]

# Unsteady velocity field
bool_unsteady = True # bool

# Defined domain
defined_domain = np.isfinite(U).astype(int) # array (NY, NX, NZ)

## Compute meshgrid of dataset
X, Y, Z = np.meshgrid(x, y, z) # array (NY, NX), array (NY, NX, NZ)

## Resolution of meshgrid
dx_data = X[0,1,0]-X[0,0,0] # float
dy_data = Y[1,0,0]-Y[0,0,0] # float
dz_data = Y[0,0,1]-Z[0,0,0] # float

delta = [dx_data, dy_data, dx_data] # list (3, )

# Interpolate velocity

In order to evaluate the velocity field at arbitrary locations, we interpolate the discrete velocity data. The interpolation with respect to time and space is linear.

In [4]:
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady

# Set nan values to zero (in case there are any) so that we can apply interpolant. 
# Interpolant does not work if the array contains nan values. 
U[np.isnan(U)] = 0
V[np.isnan(V)] = 0
W[np.isnan(W)] = 0

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_unsteady(X, Y, Z, U, V, W, time_data)

Interpolant_u = Interpolant[0] # RectangularBivariateSpline-object
Interpolant_v = Interpolant[1] # RectangularBivariateSpline-object
Interpolant_w = Interpolant[2] # RectangularBivariateSpline-object

# Compute gradient of the flow map $ \nabla F_{t_0}^t(\mathbf{x}_0) $

In [5]:
# Import function to compute gradient of flow map
from ipynb.fs.defs.gradient_flowmap import gradient_flowmap

# import package used to suppress output of integration of flow map. 
from IPython.utils import io

# Initial time
t0 = 0

# Final time
tN = 10

# Time step-size (in days)
dt = .1

# NOTE: For computing the backward trajectories set: tN < t0 and dt < 0.

time = np.arange(t0, tN+dt, dt) # len(time) = N

# auxiliary grid
aux_grid = [0.01, 0.01, 0.01]

# initial x/y-coordinate
x0 = 3
y0 = 3.4
z0 = 1

#Initial conditions
X0 = np.array([x0, y0, z0]).reshape(3,-1)

# Compute gradient of flow map
with io.capture_output() as captured:
    gradFmap = gradient_flowmap(time, X0, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, aux_grid) # array (Nt, 3, 3, 1)

# gradFmap stores the gradient of the flow map for all t in [t0,tN] 
# for the trajectory starting at the initial condition defined above.
# We extract gradient of flow map over the time-interval [t0,tN]
gradFmap_t0_tN = gradFmap[-1,:,:,0]

# Singular Value Decomposition of $ \nabla F_{t_0}^t(\mathbf{x}_0) $

In [6]:
# import function to compute SVD
from ipynb.fs.defs.SVD import SVD

# import package to print latex statement
from IPython.display import display, Markdown

P, SIG, Q = SVD(gradFmap_t0_tN)

display(Markdown(
   rf"""
   
   $\nabla \mathbf{{F}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {gradFmap_t0_tN[0,0]:5.7} & {gradFmap_t0_tN[0,1]:5.7} & {gradFmap_t0_tN[0,2]:5.7} \\ {gradFmap_t0_tN[1,0]:5.7} & {gradFmap_t0_tN[1,1]:5.7} & {gradFmap_t0_tN[1,2]:5.7} \\ {gradFmap_t0_tN[2,0]:5.7} & {gradFmap_t0_tN[2,1]:5.7} & {gradFmap_t0_tN[2,2]:5.7} \end{{pmatrix}} $
   
   Dominant left singular vector of $\nabla F_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {P[0,0]:5.7} \\ {P[1,0]:5.7} \\ {P[2,0]:5.7} \end{{pmatrix}} $
   
   Intermediate left singular vector of $\nabla F_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {P[0,1]:5.7} \\ {P[1,1]:5.7} \\ {P[2,1]:5.7} \end{{pmatrix}} $
   
   Weak left singular vector of $\nabla F_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {P[0,2]:5.7} \\ {P[1,2]:5.7} \\ {P[2,2]:5.7} \end{{pmatrix}} $
   
   Dominant right singular vector of $\nabla F_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {Q[0,0]:5.7} \\ {Q[1,0]:5.7} \\ {Q[2,0]:5.7} \end{{pmatrix}} $
   
   Intermediate right singular vector of $\nabla F_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {Q[0,1]:5.7} \\ {Q[1,1]:5.7} \\ {Q[2,1]:5.7} \end{{pmatrix}} $
   
   Weak right singular vector of $\nabla F_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {Q[0,2]:5.7} \\ {Q[1,2]:5.7} \\ {Q[2,2]:5.7} \end{{pmatrix}} $
   
   Dominant singular value of $\nabla F_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $ {SIG[0,0]:5.7} $
   
   Intermediate singular value of $\nabla F_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $ {SIG[1,1]:5.7} $
   
   Weak singular value of $\nabla F_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $ {SIG[2,2]:5.7} $
   
   """))


   
   $\nabla \mathbf{F}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} -53.03016 & -11.45435 & -23.99339 \\ 25.98287 & 4.915627 & 11.39238 \\ -25.7069 & -6.023739 & -12.48633 \end{pmatrix} $
   
   Dominant left singular vector of $\nabla F_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} -0.8226487 \\ 0.3991653 \\ -0.4048658 \end{pmatrix} $
   
   Intermediate left singular vector of $\nabla F_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 0.05147351 \\ -0.6568884 \\ -0.7522287 \end{pmatrix} $
   
   Weak left singular vector of $\nabla F_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} -0.5662152 \\ -0.6396598 \\ 0.5198419 \end{pmatrix} $
   
   Dominant right singular vector of $\nabla F_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 0.8931361 \\ 0.1917038 \\ 0.4068875 \end{pmatrix} $
   
   Intermediate right singular vector of $\nabla F_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} -0.4246118 \\ 0.6577576 \\ 0.6221413 \end{pmatrix} $
   
   Weak right singular vector of $\nabla F_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 0.1483665 \\ 0.7284261 \\ -0.6688668 \end{pmatrix} $
   
   Dominant singular value of $\nabla F_{t_0}^{t_N}(\mathbf{x}_0)$: $ 72.1105 $
   
   Intermediate singular value of $\nabla F_{t_0}^{t_N}(\mathbf{x}_0)$: $ 1.083401 $
   
   Weak singular value of $\nabla F_{t_0}^{t_N}(\mathbf{x}_0)$: $ 0.2881621 $
   
   

# References

[1] Notebook 2.3.2. in "Transport Barriers and Coherent Structures in Flow Data" by Prof. George Haller.